In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
import math

pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 1000)


folder_path = 'data/2022_23_5_shotchart.csv'

df = pd.read_csv(folder_path)



In [24]:
import io
import pandas as pd

grouped_data = []
start_season = 14
end_season = 22
group_size = 5
for season in range(start_season, end_season+1):
    # Read in the CSV file for the season
    df = pd.read_csv(f'data/20{season}_{season+1}_{group_size}_shotchart.csv')

    season_name = f'20{season}-{season+1}'

    # Group by the group id and group name

    for (group_id, group_name), group_df in df.groupby(['GROUP_ID', 'GROUP_NAME']):
        # Only select loc_x, loc_y, and shot_made_flag columns for each shot
        shots = group_df[['LOC_X', 'LOC_Y', 'SHOT_MADE_FLAG']].to_dict(orient='records')
        season = season_name
        lineup_size = 5
        grouped_data.append((group_id, group_name, season, lineup_size, shots))

print(grouped_data[-1])
print(len(grouped_data))

('-2544-203076-1626156-1629020-1630559-', 'L. James - A. Davis - D. Russell - J. Vanderbilt - A. Reaves', '2022-23', 5, [{'LOC_X': -225, 'LOC_Y': 102, 'SHOT_MADE_FLAG': 0}, {'LOC_X': -40, 'LOC_Y': 157, 'SHOT_MADE_FLAG': 1}, {'LOC_X': -31, 'LOC_Y': 67, 'SHOT_MADE_FLAG': 0}, {'LOC_X': -10, 'LOC_Y': 29, 'SHOT_MADE_FLAG': 0}, {'LOC_X': -11, 'LOC_Y': 47, 'SHOT_MADE_FLAG': 0}, {'LOC_X': 2, 'LOC_Y': 279, 'SHOT_MADE_FLAG': 0}, {'LOC_X': 0, 'LOC_Y': 10, 'SHOT_MADE_FLAG': 1}, {'LOC_X': 20, 'LOC_Y': 5, 'SHOT_MADE_FLAG': 0}, {'LOC_X': 58, 'LOC_Y': 147, 'SHOT_MADE_FLAG': 1}, {'LOC_X': -238, 'LOC_Y': 6, 'SHOT_MADE_FLAG': 1}, {'LOC_X': 1, 'LOC_Y': 4, 'SHOT_MADE_FLAG': 1}, {'LOC_X': 0, 'LOC_Y': 2, 'SHOT_MADE_FLAG': 1}, {'LOC_X': 161, 'LOC_Y': 62, 'SHOT_MADE_FLAG': 0}, {'LOC_X': 210, 'LOC_Y': 142, 'SHOT_MADE_FLAG': 1}, {'LOC_X': -1, 'LOC_Y': 7, 'SHOT_MADE_FLAG': 0}, {'LOC_X': -208, 'LOC_Y': 148, 'SHOT_MADE_FLAG': 0}, {'LOC_X': 4, 'LOC_Y': 31, 'SHOT_MADE_FLAG': 0}, {'LOC_X': 0, 'LOC_Y': 0, 'SHOT_MADE_FL

In [26]:
# Convert all tuples to dictionaries
grouped_data_dicts = []
for tup in grouped_data:
    shots_lower_case = [{k.lower(): v for k, v in shot.items()} for shot in tup[4]]
    grouped_dict = {
        "group_id": tup[0],
        "group_name": tup[1],
        "season": tup[2],
        "lineup_size": tup[3],
        "shots": shots_lower_case
    }
    grouped_data_dicts.append(grouped_dict)


In [27]:
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

# Define the schema for insertion
schema = [
    bigquery.SchemaField("group_id", "STRING"),
    bigquery.SchemaField("group_name", "STRING"),
    bigquery.SchemaField("season", "STRING"),
    bigquery.SchemaField("lineup_size", "INT64"),
    bigquery.SchemaField("shots", "RECORD", mode="REPEATED", fields=[
        bigquery.SchemaField("loc_x", "INT64"),
        bigquery.SchemaField("loc_y", "INT64"),
        bigquery.SchemaField("shot_made_flag", "INT64")
    ])
]


# Check if the table exists
dataset_ref = client.dataset('lineup_data')
table_ref = dataset_ref.table('lineup_shots')
try:
    table = client.get_table(table_ref)
except Exception as e:
    print(f"Error: {e}")
    # Handle the error, e.g., create the table if it doesn't exist
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)
    print("Table created.")

custom_timeout = 60  # 1 minute
batch_size = 50  # Adjust as needed

# single_row = [{
#     "group_id": "1",
#     "group_name": "Group 1",
#     "season": "2018-19",
#     "lineup_size": 5,
#     "shots": [
#         {"loc_x": 1, "loc_y": 2, "shot_made_flag": 1},
#         {"loc_x": 3, "loc_y": 4, "shot_made_flag": 0}
#     ]
# }]

print(len(grouped_data_dicts))
for i in range(0, len(grouped_data_dicts), batch_size):
    batch = grouped_data_dicts[i:i + batch_size]
    errors = client.insert_rows_json(table, batch, timeout=custom_timeout)
    if errors:
        print(f"Errors in batch {i//batch_size + 1}:", errors)
    else:
        print(f"Batch {i//batch_size + 1} inserted successfully!")

print('done!')


Error: 404 GET https://bigquery.googleapis.com/bigquery/v2/projects/nba5man/datasets/lineup_data/tables/lineup_shots?prettyPrint=false: Not found: Table nba5man:lineup_data.lineup_shots
Table created.
2785
Batch 1 inserted successfully!
Batch 2 inserted successfully!
Batch 3 inserted successfully!
Batch 4 inserted successfully!
Batch 5 inserted successfully!
Batch 6 inserted successfully!
Batch 7 inserted successfully!
Batch 8 inserted successfully!
Batch 9 inserted successfully!
Batch 10 inserted successfully!
Batch 11 inserted successfully!
Batch 12 inserted successfully!
Batch 13 inserted successfully!
Batch 14 inserted successfully!
Batch 15 inserted successfully!
Batch 16 inserted successfully!
Batch 17 inserted successfully!
Batch 18 inserted successfully!
Batch 19 inserted successfully!
Batch 20 inserted successfully!
Batch 21 inserted successfully!
Batch 22 inserted successfully!
Batch 23 inserted successfully!
Batch 24 inserted successfully!
Batch 25 inserted successfully!
Bat

In [11]:
import json 

# Convert the grouped_data to a DataFrame
df = pd.DataFrame(grouped_data, columns=['group_id', 'group_name', 'season', 'lineup_size', 'shots'])

# Convert the 'shots' column to a string representation
df['shots'] = df['shots'].apply(json.dumps)

# Export the DataFrame to a CSV
df.to_csv('grouped_shots.csv', index=False)


In [30]:
layout = go.Layout(
    title="Shot Accuracy and Frequency for Most Common Lineup",

    autosize=False,
    width=600,
    height=650,
    showlegend=False,
)

custom_colorscale = [
    [0, '#000000'],   # Black
    [1, '#43FF33']    # Green
]